# Ceneo scraper

## loading lirbraries

In [6]:
import requests
from bs4 import BeautifulSoup

## Sending request to Ceneo.pl server

In [5]:
product_id = '97571150;02517'
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)


In [ ]:
page_dom = BeautifulSoup(response.text , 'html.parser')
opinions = page_dom.select('.js_product-review')
opinion = page_dom.select_one('.js_product-review')
print(opinions)

##

|Component|Selector|Variable|
|---------|--------|--------|
|Opinion ID|page_dom['data-entry-id']|opinion_id|
|opinion’s author|page_dom.select('span.user-post__author-name')|author|
|author’s recommendation|page_dom.select(".user-post__author-recomendation > em")|reccomendation|
|score expressed in number of stars|page_dom.select(".user-post__score-count")|score|
|opinion’s content|page_dom.select(".user-post__text")|content|
|list of product advantages|page_dom.select(".review-feature__title--positives ~ review-feature__item")|pros|
|list of product disadvantages|page_dom.select(".review-feature__title--negatives ~ review-feature__item"")|cons|
|how many users think that opinion was helpful|span[id^="votes-yes"]|helpful|
|how many users think that opinion was unhelpful|span[id^="votes-no"]|unhelpful|
|publishing date|('span.user-post__published > time:nth-child(1)['datetime']')|publish_date|
|purchase date|('span.user-post__published > time:nth-child(2)['datetime']')|purchase_date|

In [ ]:
opinion_id = opinion['data-entry-id']
author = page_dom.select('span.user-post__author-name')
reccomendation = page_dom.select(".user-post__author-recomendation > em")
score = page_dom.select("span.user-post__score-count")
content = page_dom.select(".user-post__text")
pros = page_dom.select(".review-feature__title--positives ~ review-feature__item")
cons = page_dom.select(".review-feature__title--negatives ~ review-feature__item")
helpful = page_dom.select('span[id^="votes-yes"]')